# The setup

In [ ]:
#Define the variables.
data_directory_clean =  'Files/sampledata/Data/Wide World Importers DW/Clean/'
azure_sql_url = "jdbc:sqlserver://servername.database.windows.net"
database_name = "WideWorldImportersDW"
db_username = ''
db_password = ''

# Process the data

In [ ]:
def fn_write_data (table_name, query, number_of_partitions):
    
    destination_directory_delta = f'{data_directory_clean}/Delta/{table_name}'
    destination_directory_json = f'{data_directory_clean}/JSON/{table_name}'
    destination_directory_parquet = f'{data_directory_clean}/Parquet/{table_name}'

    #Read the files.
    df = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", azure_sql_url) \
        .option("query", query) \
        .option("databaseName", database_name) \
        .option("username", db_username) \
        .option("password", db_password) \
        .option("encrypt", "true") \
        .option("hostNameInCertificate", "*.database.windows.net") \
        .load()

    print ('Source File: ' + table_name)

    #Create the delta table.
    df.write.mode('overwrite').option('overwriteSchema', 'true').format("Delta").save(destination_directory_delta)
    print ('Creating Delta Table Complete: ' + table_name)

    #Create the JSON files.
    if number_of_partitions == 0:
        df.write.mode('overwrite').json(destination_directory_json)
    else:
        df.repartition(number_of_partitions).write.mode('overwrite').json(destination_directory_json)
    print ('Creating JSON File Complete: ' + table_name)
    
    #Create the Parquet files.
    if number_of_partitions == 0:
        df.write.mode('overwrite').parquet(destination_directory_parquet)
    else:
        df.repartition(number_of_partitions).write.mode('overwrite').parquet(destination_directory_parquet)
    print ('Creating Parquet File Complete: ' + table_name)
    print ('')

In [ ]:
table_list = [("DimCity", "SELECT \
    [City Key] AS [CityKey], \
	[WWI City ID] AS [WWICityID], \
	[City] AS [City], \
	[State Province] AS [StateProvince], \
	[Country] AS [Country], \
	[Continent] AS [Continent], \
	[Sales Territory] AS [SalesTerritory], \
	[Region] AS [Region], \
	[Subregion] AS [Subregion], \
	CONVERT(VARCHAR(50), [Location]) AS [Location], \
	[Latest Recorded Population] AS [LatestRecordedPopulation], \
	[Valid From] AS [ValidFrom], \
	[Valid To] AS [ValidTo], \
	[Lineage Key] AS [LineageKey] \
FROM [Dimension].[City]"),
    ("DimCustomer", "SELECT \
	[Customer Key] AS [CustomerKey],\
	[WWI Customer ID] AS [WWICustomerID],\
	[Customer] AS [Customer],\
	[Bill To Customer] AS [BillToCustomer],\
	[Category] AS [Category],\
	[Buying Group] AS [BuyingGroup],\
	[Primary Contact] AS [PrimaryContact],\
	[Postal Code] AS [PostalCode],\
	[Valid From] AS [ValidFrom],\
	[Valid To] AS [ValidTo],\
	[Lineage Key] AS [LineageKey]\
FROM [Dimension].[Customer]"),
    ("DimDate", "SELECT \
	[Date] AS [Date],\
	[Day Number] AS [DayNumber],\
	[Day] AS [Day],\
	[Month] AS [Month],\
	[Short Month] AS [ShortMonth],\
	[Calendar Month Number] AS [CalendarMonthNumber],\
	[Calendar Month Label] AS [CalendarMonthLabel],\
	[Calendar Year] AS [CalendarYear],\
	[Calendar Year Label] AS [CalendarYearLabel],\
	[Fiscal Month Number] AS [FiscalMonthNumber],\
	[Fiscal Month Label] AS [FiscalMonthLabel],\
	[Fiscal Year] AS [FiscalYear],\
	[Fiscal Year Label] AS [FiscalYearLabel],\
	[ISO Week Number] AS [ISOWeekNumber]\
FROM [Dimension].[Date]"),
    ("DimEmployee", "SELECT \
	[Employee Key] AS [EmployeeKey],\
	[WWI Employee ID] AS [WWIEmployeeID],\
	[Employee] AS [Employee],\
	[Preferred Name] AS [PreferredName],\
	[Is Salesperson] AS [IsSalesperson],\
	/*[Photo] AS [Photo],*/\
	[Valid From] AS [ValidFrom],\
	[Valid To] AS [ValidTo],\
	[Lineage Key] AS [LineageKey]\
FROM [Dimension].[Employee]"),
    ("DimPaymentMethod", "SELECT \
	[Payment Method Key] AS [PaymentMethodKey],\
	[WWI Payment Method ID] AS [WWIPaymentMethodID],\
	[Payment Method] AS [PaymentMethod],\
	[Valid From] AS [ValidFrom],\
	[Valid To] AS [ValidTo],\
	[Lineage Key] AS [LineageKey]\
FROM [Dimension].[Payment Method]"),
    ("DimStockItem", "SELECT \
	[Stock Item Key] AS [StockItemKey],\
	[WWI Stock Item ID] AS [WWIStockItemID],\
	[Stock Item] AS [StockItem],\
	[Color] AS [Color],\
	[Selling Package] AS [SellingPackage],\
	[Buying Package] AS [BuyingPackage],\
	[Brand] AS [Brand],\
	[Size] AS [Size],\
	[Lead Time Days] AS [LeadTimeDays],\
	[Quantity Per Outer] AS [QuantityPerOuter],\
	[Is Chiller Stock] AS [IsChillerStock],\
	[Barcode] AS [Barcode],\
	[Tax Rate] AS [TaxRate],\
	[Unit Price] AS [UnitPrice],\
	[Recommended Retail Price] AS [RecommendedRetailPrice],\
	[Typical Weight Per Unit] AS [TypicalWeightPerUnit],\
	/*[Photo] AS [Photo],*/\
	[Valid From] AS [ValidFrom],\
	[Valid To] AS [ValidTo],\
	[Lineage Key] AS [LineageKey]\
FROM [Dimension].[Stock Item]"),
    ("DimSupplier", "SELECT \
	[Supplier Key] AS [SupplierKey],\
	[WWI Supplier ID] AS [WWISupplierID],\
	[Supplier] AS [Supplier],\
	[Category] AS [Category],\
	[Primary Contact] AS [PrimaryContact],\
	[Supplier Reference] AS [SupplierReference],\
	[Payment Days] AS [PaymentDays],\
	[Postal Code] AS [PostalCode],\
	[Valid From] AS [ValidFrom],\
	[Valid To] AS [ValidTo],\
	[Lineage Key] AS [LineageKey]\
FROM [Dimension].[Supplier]"),
    ("DimTransactionType", "SELECT \
	[Transaction Type Key] AS [TransactionTypeKey],\
	[WWI Transaction Type ID] AS [WWITransactionTypeID],\
	[Transaction Type] AS [TransactionType],\
	[Valid From] AS [ValidFrom],\
	[Valid To] AS [ValidTo],\
	[Lineage Key] AS [LineageKey]\
FROM [Dimension].[Transaction Type]"),
    ("FactMovement", "SELECT \
	[Movement Key] AS [MovementKey],\
	[Date Key] AS [DateKey],\
	[Stock Item Key] AS [StockItemKey],\
	[Customer Key] AS [CustomerKey],\
	[Supplier Key] AS [SupplierKey],\
	[Transaction Type Key] AS [TransactionTypeKey],\
	[WWI Stock Item Transaction ID] AS [WWIStockItemTransactionID],\
	[WWI Invoice ID] AS [WWIInvoiceID],\
	[WWI Purchase Order ID] AS [WWIPurchaseOrderID],\
	[Quantity] AS [Quantity],\
	[Lineage Key] AS [LineageKey]\
FROM [Fact].[Movement]"),
    ("FactOrder", "SELECT \
	[Order Key] AS [OrderKey],\
	[City Key] AS [CityKey],\
	[Customer Key] AS [CustomerKey],\
	[Stock Item Key] AS [StockItemKey],\
	[Order Date Key] AS [OrderDateKey],\
	[Picked Date Key] AS [PickedDateKey],\
	[Salesperson Key] AS [SalespersonKey],\
	[Picker Key] AS [PickerKey],\
	[WWI Order ID] AS [WWIOrderID],\
	[WWI Backorder ID] AS [WWIBackorderID],\
	[Description] AS [Description],\
	[Package] AS [Package],\
	[Quantity] AS [Quantity],\
	[Unit Price] AS [UnitPrice],\
	[Tax Rate] AS [TaxRate],\
	[Total Excluding Tax] AS [TotalExcludingTax],\
	[Tax Amount] AS [TaxAmount],\
	[Total Including Tax] AS [TotalIncludingTax],\
	[Lineage Key] AS [LineageKey]\
FROM [Fact].[Order]"),
    ("FactPurchase", "SELECT \
	[Purchase Key] AS [PurchaseKey],\
	[Date Key] AS [DateKey],\
	[Supplier Key] AS [SupplierKey],\
	[Stock Item Key] AS [StockItemKey],\
	[WWI Purchase Order ID] AS [WWIPurchaseOrderID],\
	[Ordered Outers] AS [OrderedOuters],\
	[Ordered Quantity] AS [OrderedQuantity],\
	[Received Outers] AS [ReceivedOuters],\
	[Package] AS [Package],\
	[Is Order Finalized] AS [IsOrderFinalized],\
	[Lineage Key] AS [LineageKey]\
FROM [Fact].[Purchase]"),
    ("FactSale", "SELECT \
	[Sale Key] AS [SaleKey],\
	[City Key] AS [CityKey],\
	[Customer Key] AS [CustomerKey],\
	[Bill To Customer Key] AS [BillToCustomerKey],\
	[Stock Item Key] AS [StockItemKey],\
	[Invoice Date Key] AS [InvoiceDateKey],\
	[Delivery Date Key] AS [DeliveryDateKey],\
	[Salesperson Key] AS [SalespersonKey],\
	[WWI Invoice ID] AS [WWIInvoiceID],\
	[Description] AS [Description],\
	[Package] AS [Package],\
	[Quantity] AS [Quantity],\
	[Unit Price] AS [UnitPrice],\
	[Tax Rate] AS [TaxRate],\
	[Total Excluding Tax] AS [TotalExcludingTax],\
	[Tax Amount] AS [TaxAmount],\
	[Profit] AS [Profit],\
	[Total Including Tax] AS [TotalIncludingTax],\
	[Total Dry Items] AS [TotalDryItems],\
	[Total Chiller Items] AS [TotalChillerItems],\
	[Lineage Key] AS [LineageKey]\
FROM [Fact].[Sale]"),
    ("FactStockHolding", "SELECT \
	[Stock Holding Key] AS [StockHoldingKey],\
	[Stock Item Key] AS [StockItemKey],\
	[Quantity On Hand] AS [QuantityOnHand],\
	[Bin Location] AS [BinLocation],\
	[Last Stocktake Quantity] AS [LastStocktakeQuantity],\
	[Last Cost Price] AS [LastCostPrice],\
	[Reorder Level] AS [ReorderLevel],\
	[Target Stock Level] AS [TargetStockLevel],\
	[Lineage Key] AS [LineageKey]\
FROM [Fact].[Stock Holding]"),
    ("FactTransaction", "SELECT \
	[Transaction Key] AS [TransactionKey],\
	[Date Key] AS [DateKey],\
	[Customer Key] AS [CustomerKey],\
	[Bill To Customer Key] AS [BillToCustomerKey],\
	[Supplier Key] AS [SupplierKey],\
	[Transaction Type Key] AS [TransactionTypeKey],\
	[Payment Method Key] AS [PaymentMethodKey],\
	[WWI Customer Transaction ID] AS [WWICustomerTransactionID],\
	[WWI Supplier Transaction ID] AS [WWISupplierTransactionID],\
	[WWI Invoice ID] AS [WWIInvoiceID],\
	[WWI Purchase Order ID] AS [WWIPurchaseOrderID],\
	[Supplier Invoice Number] AS [SupplierInvoiceNumber],\
	[Total Excluding Tax] AS [TotalExcludingTax],\
	[Tax Amount] AS [TaxAmount],\
	[Total Including Tax] AS [TotalIncludingTax],\
	[Outstanding Balance] AS [OutstandingBalance],\
	[Is Finalized] AS [IsFinalized],\
	[Lineage Key] AS [LineageKey]\
FROM [Fact].[Transaction]")
    ]

In [ ]:
for table in table_list:
    fn_write_data (table[0], table[1], 0)